# Using EDSL to sense check data
This notebook provides example code for sense checking survey data using [EDSL](https://docs.expectedparrot.com), an open-source library for simulating surveys, experiments and market research with AI agents and large language models. 

## Contents
Using a set of responses to a survey about online marketplaces as an example, we demonstrate EDSL methods for: 

1. Evaluating survey questions (e.g., for clarity and improvements)
2. Analyzing each respondent's set of answers (e.g., to summarize or identify sentiment, themes, etc.)
3. Reviewing each answer individually (e.g., to evaluate its relevance or usefulness)

## Coop
We also show how to post EDSL questions, surveys, results and notebooks (like this one) to [Coop: an integrated platform for creating and sharing LLM-based research](https://www.expectedparrot.com/explore). 

## How EDSL works
EDSL is a flexible library that can be used to perform a broad variety of research tasks. A typical workflow consists of the following steps:

* Construct questions  
* Add data to the questions (e.g., for data labeling tasks)
* Use an AI agent to answer the questions
* Select a language model to generate the answers
* Analyze results in a formatted dataset

## Technical setup
Before running the code below please ensure that you have completed setup:

* [Install](https://docs.expectedparrot.com/en/latest/installation.html) the EDSL library.
* Create a [Coop account](https://www.expectedparrot.com/login) and activate [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) OR store your own [API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models that you want to use.

Our [Starter Tutorial](https://docs.expectedparrot.com/en/latest/starter_tutorial.html) provides examples of EDSL basic components. 

## Example data
Our example data is a CSV consisting of several questions and a few rows of responses.
Here we store it at the Coop and then re-import it.

To post an object:

    from edsl import FileStore
    
    fs = FileStore("marketplace_survey_results.csv")
    fs.push(
        description = "mock marketplace survey results",
        alias = "mock-marketplace-survey-results",
        visibility = "public"
    )

This returns details of the object we can use to retrieve it:

    {'description': 'mock marketplace survey results',
     'object_type': 'scenario',
     'url': 'https://www.expectedparrot.com/content/2e56db7a-27de-4c40-bb72-36a5b1d21e6a',
     'alias_url': 'https://www.expectedparrot.com/content/RobinHorton/mock-marketplace-survey-results',
     'uuid': '2e56db7a-27de-4c40-bb72-36a5b1d21e6a',
     'version': '0.1.62.dev1',
     'visibility': 'public'}

In [1]:
from edsl import FileStore

In [2]:
csv_file = FileStore.pull("https://www.expectedparrot.com/content/RobinHorton/mock-marketplace-survey-results")

## Creating questions about the data
There are many questions we might want to ask about the data, such as:

* Does this survey question have any logical or syntactical problems? {{ *question* }}
* What is the overall sentiment of this respondent's answers? {{ *responses* }}
* Is this answer responsive to the question that was asked? {{ *question* }} {{ *answer* }}

## Question types
EDSL comes with many common question types that we can select from based on the form of the response that we want to get back from the model: multiple choice, checkbox, linear scale, free text, etc. [Learn more about EDSL question types](https://docs.expectedparrot.com/en/latest/questions.html).

Here we construct `Question` objects for the questions that we want to ask about the data, using `{{ placeholders }}` for the information that we will add to the questions in the steps that follow:

In [3]:
from edsl import QuestionFreeText, QuestionMultipleChoice, QuestionYesNo

In [4]:
q_logic = QuestionFreeText(
    question_name = "logic",
    question_text = """
    Describe any logical or syntactical problems in the following survey question: 
    {{ scenario.question }}
    """
)

In [5]:
q_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = """
    Identify the overall sentiment of this respondent's survey answers: 
    {{ scenario.responses }}
    """,
    question_options = ["Very unsatisfied", "Somewhat unsatisfied", "Somewhat satisfied", "Very satisfied"]
)

In [6]:
q_responsive = QuestionYesNo(
    question_name = "responsive",
    question_text = """
    Is this answer responsive to the question that was asked? 
    Question: {{ scenario.question }} 
    Answer: {{ scenario.answer }}
    """
)

## Adding survey data to the questions
Next we'll add our data to our questions. This can be done efficiently by creating a `ScenarioList` representing the data. The individual `Scenario` objects in the list can be constructed in a variety of ways depending on the information that we want to include in a particular question.

We start by calling the `from_source()` method to create a `ScenarioList` for the data in its original form. We can see that this generates a `Scenario` dictionary for each respondent's set of answers with key/value pairs for the individual questions and answers: 

In [7]:
from edsl import ScenarioList

In [8]:
sl = ScenarioList.from_source("csv", csv_file.to_tempfile()) # equivalent to importing a local file
sl

,Respondent ID,What do you like most about using our online marketplace?,What is one feature you would like to see added to improve your shopping experience?,Can you describe a recent experience where you were dissatisfied with our service?,How do you feel about the current product search and filtering options?,Is there anything else you would like to share about your experience with us?
0,101,The wide variety of products and the ease of use.,It would be great to have a personalized recommendation system based on my browsing history.,"I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.",The search and filtering options are intuitive and work well for me.,"No, keep up the great work!"
1,102,I enjoy the simplicity of the interface.,A feature that helps compare similar products side by side would be useful.,No complaints here.,I find the product search to be pretty effective.,I think the sky is a beautiful shade of purple today.
2,103,The platform is user-friendly and offers a vast selection of products.,Would love to see an option to save and compare different products.,"My delivery was late by a few days, which was frustrating.",It’s okay.,No.


## Evaluating the questions
For our first question we want to create a `Scenario` for each survey question:

In [9]:
from edsl import QuestionFreeText, Survey

q_logic = QuestionFreeText(
    question_name = "logic",
    question_text = """
    Describe any logical or syntactical problems in the following survey question: 
    {{ scenario.question }}
    """
)

q_improved = QuestionFreeText(
    question_name = "improved",
    question_text = """
    You were previously asked to describe any problems with the following survey question:
    {{ scenario.question }}
    You answered: '{{ logic.answer }}'
    Now please draft an improved version of this survey question. 
    Return only the revised question text.
    """
)

survey = Survey(questions = [q_logic, q_improved])

The survey questions are the `parameters` of the `ScenarioList` created above:

In [10]:
questions = list(sl.parameters)
questions

['Respondent ID',
 'What do you like most about using our online marketplace?',
 'What is one feature you would like to see added to improve your shopping experience?',
 'Is there anything else you would like to share about your experience with us?',
 'How do you feel about the current product search and filtering options?',
 'Can you describe a recent experience where you were dissatisfied with our service?']

We can pass them to the `from_source()` method to create a new `ScenarioList`, specifying that the key for each `Scenario` will be `question` in order to match the parameter of our logic question:

In [11]:
sl_questions = ScenarioList.from_source("list", "question", questions)
sl_questions

,question
0,Respondent ID
1,What do you like most about using our online marketplace?
2,What is one feature you would like to see added to improve your shopping experience?
3,Is there anything else you would like to share about your experience with us?
4,How do you feel about the current product search and filtering options?
5,Can you describe a recent experience where you were dissatisfied with our service?


We select a model to use, and then add the scenarios to the survey when we run it:

In [12]:
from edsl import Model

m = Model("gemini-1.5-flash", service_name = "google")

In [13]:
results = survey.by(sl_questions).by(m).run()

This generates a dataset of `Results` that we can access with built-in methods for analysis:

In [14]:
results.select("question", "logic", "improved")

,scenario.question,answer.logic,answer.improved
0,Respondent ID,"""Respondent ID"" is not a survey *question*; it's a field for identifying the respondent's unique entry in the dataset. There's no logical or syntactical problem *within* the term itself, but its placement as if it were a question is problematic. The issue is its function: it's not something a respondent *answers*, it's something assigned to them by the survey system. Including it as if it were a question is a design flaw.","(No change needed; remove ""Respondent ID"" from the survey entirely.)"
1,What do you like most about using our online marketplace?,"The question ""What do you like most about using our online marketplace?"" has a few potential problems: * **Leading Question (subtle):** While not overtly leading, it presupposes the respondent *likes* something about the marketplace. Someone with a negative experience might struggle to answer honestly, potentially leading to biased results or a lack of negative feedback. A neutral question would be better. * **Open-ended and difficult to analyze:** The open-ended nature makes it hard to quantify and analyze the responses. You'll get a wide variety of answers requiring significant manual coding and categorization, making it time-consuming and prone to subjective interpretation. This is less of a logical problem and more of a practical one for data analysis. * **Ambiguity on ""using"":** Does ""using"" refer to the buying process, selling process, browsing, customer service interactions, or something else? The question could be more specific to elicit more targeted and useful feedback. To improve it, consider several options: * **Multiple choice with an ""other"" option:** This allows for easier analysis and quantification while still capturing a range of opinions. * **Rating scales:** Ask respondents to rate different aspects of the marketplace (e.g., ease of use, selection, customer service) on a scale. * **A more neutral open-ended question:** ""What is your overall experience with our online marketplace?"" or ""What could we improve about our online marketplace?"" These avoid the presumption of positive feelings. Even better would be to break this down into more specific questions about different aspects of the marketplace.",How would you rate your overall experience with our online marketplace?
2,What is one feature you would like to see added to improve your shopping experience?,"The question has a few potential problems: * **Ambiguity of ""feature"":** The term ""feature"" is quite broad. A respondent might interpret it as a website feature (e.g., better search functionality), a store feature (e.g., more parking), a product feature (e.g., sustainable packaging), or a service feature (e.g., faster delivery). This lack of specificity makes the responses difficult to categorize and analyze meaningfully. * **Leading question (slightly):** While not overtly leading, the phrasing implies that there *is* something to improve. Some respondents might have a perfectly satisfactory shopping experience and struggle to answer, leading to potentially biased results or low response rates. A neutral phrasing would be preferable. * **Limited to one feature:** Respondents might have multiple suggestions. Limiting them to one forces prioritization which might not accurately reflect their preferences. They might choose a less important feature simply because it's easier to articulate than a more complex issue. * **Lack of context:** The question doesn't specify *what* shopping experience is being referred to. Is it online shopping, in-store shopping, or both? This ambiguity could lead to responses that are irrelevant or difficult to compare. In short, the question needs more precision and clarity to be effective.","Thinking about your recent online shopping experience with [Company Name], what is one specific website feature you would like to see improved or added?"
3,Is there anything else you would like to share about you

[Learn more about working with results](https://docs.expectedparrot.com/en/latest/results.html).

## Evaluating respondents' collective answers
Next we can create a `ScenarioList` for each respondent's answers to use with our question about sentiment:

In [15]:
sl

,Respondent ID,What do you like most about using our online marketplace?,What is one feature you would like to see added to improve your shopping experience?,Can you describe a recent experience where you were dissatisfied with our service?,How do you feel about the current product search and filtering options?,Is there anything else you would like to share about your experience with us?
0,101,The wide variety of products and the ease of use.,It would be great to have a personalized recommendation system based on my browsing history.,"I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.",The search and filtering options are intuitive and work well for me.,"No, keep up the great work!"
1,102,I enjoy the simplicity of the interface.,A feature that helps compare similar products side by side would be useful.,No complaints here.,I find the product search to be pretty effective.,I think the sky is a beautiful shade of purple today.
2,103,The platform is user-friendly and offers a vast selection of products.,Would love to see an option to save and compare different products.,"My delivery was late by a few days, which was frustrating.",It’s okay.,No.


In [16]:
sl_responses = ScenarioList.from_source("list", "responses", [str(x) for x in sl])
sl_responses

,responses
0,"Scenario({'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!'})"
1,"Scenario({'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feature that helps compare similar products side by side would be useful.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'No complaints here.', 'How do you feel about the current product search and filtering options?': 'I find the product search to be pretty effective.', 'Is there anything else you would like to share about your experience with us?': 'I think the sky is a beautiful shade of purple today.'})"
2,"Scenario({'Respondent ID': '103', 'What do you like most about using our online marketplace?': 'The platform is user-friendly and offers a vast selection of products.', 'What is one feature you would like to see added to improve your shopping experience?': 'Would love to see an option to save and compare different products.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'My delivery was late by a few days, which was frustrating.', 'How do you feel about the current product search and filtering options?': 'It’s okay.', 'Is there anything else you would like to share about your experience with us?': 'No.'})"


Next we add these scenarios to our sentiment question (and any others we want to add) and run it:

In [17]:
from edsl import QuestionMultipleChoice, QuestionLinearScale, Survey

q_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = """
    What is the overall sentiment of this respondent's survey answers? 
    {{ scenario.responses }}
    """,
    question_options = ["Very unsatisfied", "Somewhat unsatisfied", "Somewhat satisfied", "Very satisfied"]
)

q_recommend = QuestionLinearScale(
    question_name = "recommend",
    question_text = """
    On a scale from 1 to 5, how likely do you think this respondent is to recommend the company to a friend? 
    {{ scenario.responses }}
    """,
    question_options = [1, 2, 3, 4, 5],
    option_labels = {1:"Not at all likely", 5:"Very likely"}
)

survey = Survey(questions = [q_sentiment, q_recommend])

In [18]:
jobs = results = survey.by(sl_responses).by(m)

jobs.prompts().select("user_prompt")

,user_prompt
0,"What is the overall sentiment of this respondent's survey answers? Scenario({'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!'}) Very unsatisfied Somewhat unsatisfied Somewhat satisfied Very satisfied Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line."
1,"On a scale from 1 to 5, how likely do you think this respondent is to recommend the company to a friend? Scenario({'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!'}) 1 : Not at all likely 2 : 3 : 4 : 5 : Very likely Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line."
2,"What is the overall sentiment of this respondent's survey answers? Scenario({'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feature that helps compare similar products side by side would be useful.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'No complaints here.', 'How do you feel about the current product search and filtering options?': 'I find the product search to be pretty effective.', 'Is there anything else you would like to share about your experience with us?': 'I think the sky is a beautiful shade of purple today.'}) Very unsatisfied Somewhat unsatisfied Somewhat satisfied Very satisfied Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line."
3,"On a scale from 1 to 5, how likely do you think this respondent is to recommend the company to a friend? Scenario({'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feature that helps compare similar products side by side would be useful.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'No complaints here.', 'How do you feel about the current product search and filtering options?': 'I find the product search to be pretty effective.', 'Is there anything else you would like to s

In [19]:
results = survey.by(sl_responses).by(m).run()

In [20]:
results.select("responses", "sentiment", "recommend")

,scenario.responses,answer.sentiment,answer.recommend
0,"Scenario({'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!'})",Very satisfied,4
1,"Scenario({'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feature that helps compare similar products side by side would be useful.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'No complaints here.', 'How do you feel about the current product search and filtering options?': 'I find the product search to be pretty effective.', 'Is there anything else you would like to share about your experience with us?': 'I think the sky is a beautiful shade of purple today.'})",Very satisfied,4
2,"Scenario({'Respondent ID': '103', 'What do you like most about using our online marketplace?': 'The platform is user-friendly and offers a vast selection of products.', 'What is one feature you would like to see added to improve your shopping experience?': 'Would love to see an option to save and compare different products.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'My delivery was late by a few days, which was frustrating.', 'How do you feel about the current product search and filtering options?': 'It’s okay.', 'Is there anything else you would like to share about your experience with us?': 'No.'})",Somewhat satisfied,3


## Evaluating individual answers
Next we create a `ScenarioList` for each individual question and answer to use with our question about the responsiveness of each answer. We can use the `unpivot()` method to expand the scenarios by desired identifiers (e.g., respondent ID):

In [21]:
sl_qa = sl.unpivot(id_vars = ["Respondent ID"])
sl_qa

,Respondent ID,variable,value
0,101,What do you like most about using our online marketplace?,The wide variety of products and the ease of use.
1,101,What is one feature you would like to see added to improve your shopping experience?,It would be great to have a personalized recommendation system based on my browsing history.
2,101,Can you describe a recent experience where you were dissatisfied with our service?,"I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it."
3,101,How do you feel about the current product search and filtering options?,The search and filtering options are intuitive and work well for me.
4,101,Is there anything else you would like to share about your experience with us?,"No, keep up the great work!"
5,102,What do you like most about using our online marketplace?,I enjoy the simplicity of the interface.
6,102,What is one feature you would like to see added to improve your shopping experience?,A feature that helps compare similar products side by side would be useful.
7,102,Can you describe a recent experience where you were dissatisfied with our service?,No complaints here.
8,102,How do you feel about the current product search and filtering options?,I find the product search to be pretty effective.
9,102,Is there anything else you would like to share about your experience with us?,I think the sky is a beautiful shade of purple today.


We can call the `rename()` method to rename the keys as desired to match our question parameters syntax:

In [22]:
sl_qa = sl_qa.rename({"Respondent ID": "id", "variable": "question", "value": "answer"})
sl_qa

,id,question,answer
0,101,What do you like most about using our online marketplace?,The wide variety of products and the ease of use.
1,101,What is one feature you would like to see added to improve your shopping experience?,It would be great to have a personalized recommendation system based on my browsing history.
2,101,Can you describe a recent experience where you were dissatisfied with our service?,"I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it."
3,101,How do you feel about the current product search and filtering options?,The search and filtering options are intuitive and work well for me.
4,101,Is there anything else you would like to share about your experience with us?,"No, keep up the great work!"
5,102,What do you like most about using our online marketplace?,I enjoy the simplicity of the interface.
6,102,What is one feature you would like to see added to improve your shopping experience?,A feature that helps compare similar products side by side would be useful.
7,102,Can you describe a recent experience where you were dissatisfied with our service?,No complaints here.
8,102,How do you feel about the current product search and filtering options?,I find the product search to be pretty effective.
9,102,Is there anything else you would like to share about your experience with us?,I think the sky is a beautiful shade of purple today.


In [23]:
from edsl import QuestionYesNo

q_responsive = QuestionYesNo(
    question_name = "responsive",
    question_text = """
    Is this answer responsive to the question that was asked? 
    Question: {{ scenario.question }} 
    Answer: {{ scenario.answer }}
    """
)

In [24]:
results = q_responsive.by(sl_qa).by(m).run()

In [25]:
(
    results
    .filter("responsive == 'No'")
    .select("id", "question", "answer")
)

,scenario.id,scenario.question,scenario.answer
0,102,Can you describe a recent experience where you were dissatisfied with our service?,No complaints here.
1,102,How do you feel about the current product search and filtering options?,I find the product search to be pretty effective.
2,102,Is there anything else you would like to share about your experience with us?,I think the sky is a beautiful shade of purple today.
3,103,How do you feel about the current product search and filtering options?,It’s okay.


## Uploading content to Coop
[Coop](https://www.expectedparrot.com/content/content/explore) is a new platform for creating, storing and sharing LLM-based research. It is fully integrated with EDSL, and a convenient place to post and access surveys, agents, results and notebooks. [Learn more about using Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we post the contents of this notebook:

In [26]:
from edsl import Notebook

nb = Notebook(path = "scenariolist_unpivot.ipynb")

nb.push(
    description = "ScenarioList methods for sense checking survey data", 
    alias = "scenariolist-sense-checking-survey-notebook",
    visibility = "public"
)

{'description': 'ScenarioList methods for sense checking survey data',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/08340981-2497-4be1-b9dc-be8e2f92e4ef',
 'alias_url': 'https://www.expectedparrot.com/content/RobinHorton/scenariolist-sense-checking-survey-notebook',
 'uuid': '08340981-2497-4be1-b9dc-be8e2f92e4ef',
 'version': '0.1.62.dev1',
 'visibility': 'public'}